On souhaite prédire les valeurs de Response avec un modèle Bayésien

L'objectif de ce notebook est de tester l'efficacité d'un modèle Bayésien pour prédire la catégorie à laquelle une personne sera associée en fonction des information qui le caractérisent.
<br>
Ce notebook fait suite a l'analyse exploratoire faite au lien suivant: https://www.kaggle.com/alexdarge/approche-bay-sienne

# Plan
<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Sommaire</h3>
    
<font size=+1><b>Feuille de route</b></font>
    
<font size=+1><b>Chargement des données</b></font>
* [Import des fichiers](#0)
* [Import des librairies](#1)

<font size=+1><b>Préparation avant modélisation</b></font>
* [Nettoyage de données](#2)
* [Encodage de la variable catégorielle](#3)
* [Sélection des variables pour modélisation](#4)

    
<font size=+1><b>Bayésien naïf</b></font>
* [Une 1ère prédiction](#5)
* [Modèle sans la variable Product_Info_2](#6)
* [Matrice de confusion sur le 2ème modèle](#7)
 
        
<font size=+1><b>Réduction dimensionnelles</b></font>
* [t-SNE](#8)
* [ACP](#9)

    
<font size=+1><b>Tests</b></font>
* [Tests infructueux](#10)


# Feuille de route
## Fait:
* Bayésien naif sur l'ensemble des variables
* Bayésien sur les variables numériques
* Réduction dimensionnelle avec PCA et t-SNE
* Représentation 

## A faire:
* Identifier une métrique de contrôle afin de comparer avec d'autres modèles prédictifs
* Prédiction avec les features réduites
* Améliorer la performance du modèle

<a id="0"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Import des fichiers

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="1"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Import des librairies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from category_encoders.target_encoder import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import manifold

import plotly.express as px
from sklearn.decomposition import PCA
from sklearn import decomposition




%matplotlib inline

# Préparation avant modélisation

<a id="2"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Nettoyage des données

In [ ]:
train = pd.read_csv('../input/prudential-life-insurance-assessment/train.csv.zip')
test = pd.read_csv('../input/prudential-life-insurance-assessment/test.csv.zip')

In [ ]:
# fonction données manquantes
def missing(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Pourcent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return tt
missing(train)['Pourcent'].sort_values(ascending=False)
# suppression des features données manquantes
train_modified = train[train.columns[train.isnull().mean() <= 0.75]]

In [ ]:
# drop données non renseignées
cols_with_missing = [col for col in train_modified.columns 
                                 if train_modified[col].isnull().any()]

In [ ]:
data = train_modified.copy()

<a id="3"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Encodage de la variable catégorielle

In [ ]:
train.head()

On voit que la variable Product_Info_2 est catégorielle, on s'y intéresse en vue de l'encoder numériquement

In [ ]:
print(len(train.Product_Info_2.unique()))
train.Product_Info_2.unique()

On voit que la variable Product_Info_2 est définie comme une catégorie, elle prend 19 valeurs différentes. Ces valeurs sont l'association d'une lettre et d'un chiffre. Pour l'interpréter numériquement on doit l'encoder c'est à dire créer des features supplémentaires (pour chacune des 19 valeurs). Ce variables seront des booléens qui indiquent quelle valeur de Product_Info_2 la personne a indiqué.
<br>
Par exemple un si une personne est caracterisée par la variable Product_Info_2 renseignée comme: 'D3', alors l'encodage des variables encodée sera comme suit: Product_Info_2_D3 vaudra 1 et tous les autres vaudront 0.

In [ ]:
# encodage pour la variable catégorielle
encoded_train = pd.get_dummies(train_modified)
encoded_train.sample(2)

<a id="4"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Sélection des variables pour la modélisation

In [ ]:
# suppression des colonnes données manquantes
train_modified = encoded_train.drop(cols_with_missing, axis=1)

In [ ]:
# selection de la variable cible pour la modélisation
y=train_modified.Response

In [ ]:
# imputation des données manquantes
my_imputer = SimpleImputer()
imputed_data_train = my_imputer.fit_transform(train_modified)

In [ ]:
# sélection des variables pour la modélisation
df_features = train_modified.loc[:, train_modified.columns != 'Response']

In [ ]:
# formalisation et vérification des dimensions avant modélisation
features=list(df_features.columns)
X=train_modified[features]
print(X.shape)
print(y.shape)

# Bayésien naïf

In [ ]:
# configuation du modèle
Bayes = MultinomialNB()
Bayes.fit(X,y)

<a id="5"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Un premier résultat

In [ ]:
# affichage comparatif
predict_df=pd.DataFrame(data=Bayes.predict(X))
predict_df.rename(columns={0:'Predicted'}, inplace=True)
predict_df['Response']=y
predict_df

In [ ]:
# nombre des valeurs pour la variable prédite
predict_df.Predicted.value_counts()

In [ ]:
# valeur des prédictions
sns.countplot(data=predict_df, x='Predicted').set_title("Prédiction pour chaque catégorie")


In [ ]:
# Comparaison prédiction avec les valeurs réelles
fig, axes = plt.subplots(1,2,figsize=(16,6))
fig.suptitle('Comparaison classes prédites vs classes réelles')
sns.countplot(ax=axes[0], data=predict_df, x='Predicted')
sns.countplot(ax=axes[1], data=predict_df, x='Response')
plt.show()

In [ ]:
#predict_df.countplot(x='predict_df', y=['Predicted','Response'], figsize=(10,5), grid=True)

In [ ]:
"""width = 0.3
fig, ax = plt.subplots(figsize=(12,8))
rects1 = ax.bar(x - width/2, predict_df['Response'], width)
rects2 = ax.bar(x + width/2, predict_df['Predicted'], width)

plt.title('Comparaison Response/Predicted', fontsize = 20)
plt.xlabel('Nombre', fontsize = 15)
plt.ylabel('Catégories', fontsize = 15)

ax.set_ylim(top=1)

plt.xticks(x, results_df.index)
ax.legend()
plt.grid(linestyle='dotted')
plt.show()"""

In [ ]:
# différence entre la variable prédite et Response
predict_df['diff']=abs(predict_df.Predicted-predict_df.Response)
predict_df.sample(5)

In [ ]:
# différence de classe entre prédiction et Response
predict_df['diff'].value_counts()

In [ ]:
# quantité de données différence classes prédiction/réelles
sns.countplot(data=predict_df, x='diff').set_title("Classes d'écart entre valeurs prédites et valeurs réelles")


In [ ]:
predict_df['diff'].value_counts(normalize=True)

On n'obtient que 30% de résultat exact.
On peut expliquer cela à cause de l'encodage de la variable catégorielle qui créé beaucoup de variables numérique et qui peut biaiser la prédiction.
<br>
On constate cependant qu'en cumulé on obtient près de 72% de précision à deux classes d'écart et 48% à une classe d'écart.

On ne prendra pas en considération cette variable dans le test suivant.

<a id="6"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Modèle sans la variable Product_Info_2

In [ ]:
data.head()

In [ ]:
data = data.drop('Product_Info_2', axis=1)

In [ ]:
data.head()

In [ ]:
# suppression des données non renseignées
cols_with_missing = [col for col in data.columns 
                                 if data[col].isnull().any()]

In [ ]:
data = data.drop(cols_with_missing, axis=1)

In [ ]:
y=data.Response

In [ ]:
features=list(data.columns)
X=data[features]
print(X.shape)
print(y.shape)

In [ ]:
Bayes2 = MultinomialNB()
Bayes2.fit(X,y)

In [ ]:
# affichage comparatif
predict2=pd.DataFrame(data=Bayes2.predict(X))
predict2.rename(columns={0:'Predicted'}, inplace=True)
predict2['Response']=y
predict2

In [ ]:
# nombre des valeurs pour la variable prédite
predict_df.Predicted.value_counts() 

In [ ]:
# valeur des prédictions
sns.countplot(data=predict2, x='Predicted').set_title("Prédiction pour chaque catégorie")
plt.grid(linestyle='dotted')
plt.show()

In [ ]:
# comparaison du modèle avec la valeurs réelles
fig, axes = plt.subplots(1,2,figsize=(16,6))
fig.suptitle('Comparaison classes prédites vs classes réelles')
sns.countplot(ax=axes[0], data=predict2, x='Predicted')
sns.countplot(ax=axes[1], data=predict2, x='Response')
plt.show()

In [ ]:
# différence entre la variable prédite et Response
predict2['diff']=abs(predict2.Predicted-predict2.Response)
predict2.sample(5)

In [ ]:
# différence de classe entre prédiction et Response
predict2['diff'].value_counts()

In [ ]:
# quantité de données différence classes prédiction/réelles
sns.countplot(data=predict2, x='diff').set_title("classes d'écart entre valeurs prédites et valeurs réelles")
plt.grid(linestyle='dotted')
plt.show()

In [ ]:
# pourcentage de classes écart entre prédiction et 'Response'
predict2['diff'].value_counts(normalize=True)


On a réussi à améliorer la performance de notre prédiction, en effet elle était de 30% dans notre premier résultat et est dorénavent de 36%.
<br>
De plus en cumulé à deux classe d'écart on obtient 83% de précision. C'est près de 11 points de pourcentage de plus que dans le modèle précédent.

In [ ]:
# affichage pourcentage camenbert
#predict2.plot(kind='pie', y = 'diff', legend = True)

In [ ]:
#plot = predict2.plot.pie(subplots=True, figsize=(8, 8))


<a id="7"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Matrice de confusion

In [ ]:
# matrice de confusion
confusion_matrix=confusion_matrix(predict2.Response, predict2.Predicted)

In [ ]:
print(confusion_matrix)

In [ ]:
# affichage du rapport de la matrice de confusion
print(classification_report(predict2.Response, predict2.Predicted))

In [ ]:
y_actu=predict2.Response
y_pred=predict2.Predicted

In [ ]:
y_actu=pd.Series(predict2.Response, name='Réelle')
y_pred=pd.Series(predict2.Predicted, name='Prédite')
df_confusion=pd.crosstab(y_actu, y_pred)

In [ ]:
df_confusion

In [ ]:
# normalisation de la matrice de confusion
df_conf_norm = df_confusion / df_confusion.sum(axis=1)
df_conf_norm

In [ ]:
def plot_confusion_matrix(df_confusion, title='Confusion matrix', cmap=plt.cm.Reds):
    plt.matshow(df_confusion, cmap=cmap)
    plt.title('Matrice de confusion')
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
    plt.ylabel(df_confusion.index.name)
    plt.xlabel(df_confusion.columns.name)
    plt.show()

In [ ]:
# heatmap matrice de confusion
plot_confusion_matrix(df_confusion)

In [ ]:
# heatmap matrice de confusion normalisée
plot_confusion_matrix(df_conf_norm)

# Réduction avec SNE/ACP

<a id="8"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## t-SNE

t-SNE est un algorithme de réduction de dimensions basé sur de l'apprentissage non supervisé. Il est utilisé pour de la visualisation de données ayant beaucoup de descripteurs.

Il permet de représenter les données dans un nouvel espace interprétable (2 ou 3 dimensions)
Les données proches dans l'espace original auront une probabilité élevée d'avoir une représentation proche dans le nouvel espace et à l'inverse les données éloignées ont une faible probabilité d'avoir une représentation proche dans le nouvel espace.

In [ ]:
# création d'un pipeline: évite la fuite de données
def define_preprocessor(X):
   
    # Pipeline features catégorielles
    categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), # simple imputation 
            ('target_encoder', TargetEncoder()), 
            ('scaler', StandardScaler()), # standardizsation apres encodage
            ])
    
    # pipeline features numériques
    numeric_transformer = Pipeline(steps=[
            ('imputer', IterativeImputer(max_iter=10)), 
            ('scaler', StandardScaler()), # standardisation
             ])

    # pipelines features numériques et catégorielles
    preprocessor = ColumnTransformer(transformers=[
            ('cat', categorical_transformer, list(X.select_dtypes(include=['category', 'bool']).columns)),
            ('num', numeric_transformer, list(X.select_dtypes(include='number').columns)),
            ])
    
    return preprocessor

In [ ]:
# fonction préprocessing pour le SNE
def preprocessing_tSNE(dataframe, target_name='TARGET'):
    
    X = dataframe.copy()

    # suppression lignes ou il manque la valeur cible
    X = X.dropna(subset=[target_name])

    # définition variable cible
    y = X[target_name]

    # retire variable cible des feautures interprétées
    X = X.drop(columns=[target_name])

    # applique la fonction de préprocessing
    preprocessor = define_preprocessor(X)

    # applique process
    X_std = preprocessor.fit_transform(X, y)
    
    return (X_std, y)

In [ ]:
# fonction SNE, permet aussi l'affichage
def tSNE(dataframe, target_name='TARGET'):
    
    # tritement pour tSNE
    (X_std, y) = preprocessing_tSNE(dataframe, target_name)

    # Instanciation tSNE
    tsne = manifold.TSNE(n_components=2,
                         perplexity=30,
                         n_iter=300,
                         init='pca', # initialisation avec une PCA
                         random_state=0
                        )

    # Applying tSNE
    X_projected = tsne.fit_transform(X_std) 
    
    # Affichage
    plt.figure(figsize=(14,8))

    # limites graphe
    plt.xlim(X_projected[:,0].min()*1.1, X_projected[:,0].max()*1.1)
    plt.ylim(X_projected[:,1].min()*1.1, X_projected[:,1].max()*1.1)

    # définition des axes
    plt.title("t-SNE\n", fontsize=20)
    plt.xlabel("t-SNE feature 1")
    plt.ylabel("t-SNE feature 2")

    # Def nuages de points
    sc = plt.scatter(X_projected[:,0], # x
                 X_projected[:,1], #y
                 c=y,
                 cmap=plt.cm.get_cmap('RdYlGn_r'), # couleur
                 marker='.'
        )
    
    # configuration et échelle
    cbar = plt.colorbar(sc)
    cbar.ax.get_yaxis().set_ticks([])
    cbar.ax.get_yaxis().labelpad = 15
    cbar.set_label(target_name, rotation=90)

In [ ]:
# affichage
tSNE(data, target_name='Response')

<a id="8"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## ACP

On veut déterminer un nombre minimal de composantes à partir duquel on peut considérer que l'étude prédictive est fiable à partir d'un certain seuil (que je définis ici à 90%).
Il est important de normaliser les données pour faire une PCA (pour la conservation de la distance vectorielle).
Les données fournies ont déja été normalisée il n'est donc pas nécessaire de le faire ici. 

Il faudra néanmoins faire attention aux outliers.

In [ ]:
# Preprocessing  ACP
dataframe = data
target_name = 'Response'
(X_std, y) = preprocessing_tSNE(dataframe, target_name)

# Calcul des composantes principales

n_components=2
pca = decomposition.PCA(n_components=n_components)
pca.fit(X_std)

print("Pourcentage variance expliquée par composante:", pca.explained_variance_ratio_)
print("Pourcentage total variance expliquée:", pca.explained_variance_ratio_.sum()) #  somme cumulée 

In [ ]:
# représentation de l'ACP avec deux composantes
pca=PCA(n_components=2)
components=pca.fit_transform(X_std)
fig = px.scatter(components, x=0, y=1, color=data.Response, title='Représentation PCA 2 components')
fig.show()

In [ ]:

import plotly.graph_objects as go
components=pca.fit_transform(X_std)
fig=go.Scattergl(components, x=0, y=1, color=data.Response, title='Représentation PCA 2 components')


In [ ]:
# représentation de l'ACP avec 3 composantes
pca = PCA(n_components=3)
components = pca.fit_transform(X_std)
total_var = pca.explained_variance_ratio_.sum() * 100
fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=data.Response,
    title='PCA avec 3 composantes\nTotal Explained Variance: {}%'.format(total_var),
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

In [ ]:
# fonction explicative des valeur propres pour PCA
def display_scree_plot(X_std):
    pca = decomposition.PCA()
    pca.fit(X_std)
    scree = pca.explained_variance_ratio_*100
    
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(),c="red",marker='o')
    plt.xlabel("Quantité de composantes")
    plt.ylabel("pourcentage de variance cumulée")
    plt.title("Etude du seuil de variance en fonction du nombre de composantes pour la PCA", fontsize=15)
    plt.grid(linestyle='dotted')
    plt.show(block=False)

In [ ]:
# affichage valeurs propres PCA
display_scree_plot(X_std)

In [ ]:
# fonction donne les composantes principales de l'ACP, jusqu'au seuil de variance
def PCA_features_reduction(X_std, var_threshold=0.9): 
    # PCA
    pca = decomposition.PCA()
    pca.fit(X_std)
    
    # ratio de variance expliqué pour chaque composante principale
    scree = pca.explained_variance_ratio_
    # rend le nombre de composants principaux pour atteindre les seuils de variance
    mask = scree.cumsum() > var_threshold
    nb_selected_features = len(scree[~mask]) + 1
    print("Nombre de features selectionnées:", nb_selected_features)
    
    # Calcul du ratio
    explained_variance_sum = scree.cumsum()[nb_selected_features-1]
    print("Valeur cumulée de variance expliquée:  {:.2f}%".format(explained_variance_sum*100))
    
    # projection sur les 1ers composant
    X_projected = pca.transform(X_std)[:,:nb_selected_features]
    
    return X_projected

In [ ]:
X_projected = PCA_features_reduction(X_std, var_threshold=0.9)
X_projected.shape

Le seuil de 90% de variance expliquée est atteint avec 78 composantes principales.
On avait initialement près de 120 variables.

On souhaite connaître ces 78 composantes identifiées par la PCA afin de refaire un modèle prédictif sur celles-ci.

# Tests

<a id="9"></a>
<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Sommaire</a>
## Tests infructueux

In [ ]:
train_modified2 = train[train.columns[train.isnull().mean() <= 0.75]]
train_modified2 = train.drop(cols_with_missing, axis=1)

train_modified2 = train_modified2.loc[:, train_modified2.columns != 'Response']
train_modified2 = train_modified2.loc[:, train_modified2.columns != 'Product_Info_2']


In [ ]:
train_modified2

In [ ]:
 df_features2 = train_modified2.loc[:, train_modified2.columns != 'Response']

In [ ]:
features2=list(df_features2.columns)
X2=train_modified2[features2]
print(X2.shape)
print(y.shape)

In [ ]:
'''Bayes2 = MultinomialNB()
Bayes2.fit(X2,y)'''

#### Bayes Gaussian Mixture

In [ ]:
from sklearn.mixture import BayesianGaussianMixture
Bayes_mix = BayesianGaussianMixture()
Bayes_mix.fit(X,y)



In [ ]:
Bayes_mix.fit_predict((X,y))

In [ ]:
predict_mix=pd.DataFrame(data=Bayes_mix.fit_predict(X))
predict_mix.rename(columns={0:'Predicted'}, inplace=True)
predict_mix['Response']=y
predict_mix